<a href="https://colab.research.google.com/github/lhgrandgtr/CS50x-final-project/blob/master/Churn_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()


Saving churn_dataset.csv to churn_dataset (2).csv


In [85]:
import io
#reading the dataset 
churn_data = pd.read_csv(io.BytesIO(uploaded['churn_dataset.csv']))
np.random.seed(0)

Taking a look at the dataframe to get an idea about the dataset

In [86]:
print(churn_data.head())
print(churn_data.describe())
print(churn_data.info())


   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

All the categorical data here are nominal data and 'TotalCharges' which should be numerical is an object.

In [87]:
cols_with_missing_values = [col for col in churn_data.columns if churn_data[col].isnull().any()]
print(cols_with_missing_values)

[]


There are no columns with missing values detected.


'TotalCharge' columns has float values but it's dtype is object. There are also some undetected missing values.

In [88]:
values = pd.to_numeric(churn_data['TotalCharges'], errors='coerce').notnull()
missing_values_index_list = list(churn_data[~values].index)
print(missing_values_index_list)

[488, 753, 936, 1082, 1340, 3331, 3826, 4380, 5218, 6670, 6754]


'TotalCharges' values are empty. Dropping those values.

In [89]:
cleaned_churn_data = churn_data[~churn_data.index.isin(missing_values_index_list)]

notnull_values = pd.to_numeric(cleaned_churn_data['TotalCharges'], errors='coerce').notnull()
null_list = list(cleaned_churn_data[~notnull_values].index)
print(null_list)

[]


TotalCharges columns is cleaned. Now converting the object dtype to a float dtype.

In [90]:
cleaned_churn_data.TotalCharges = cleaned_churn_data.TotalCharges.astype(float)
print(cleaned_churn_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   gender            7032 non-null   object 
 2   SeniorCitizen     7032 non-null   int64  
 3   Partner           7032 non-null   object 
 4   Dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   PhoneService      7032 non-null   object 
 7   MultipleLines     7032 non-null   object 
 8   InternetService   7032 non-null   object 
 9   OnlineSecurity    7032 non-null   object 
 10  OnlineBackup      7032 non-null   object 
 11  DeviceProtection  7032 non-null   object 
 12  TechSupport       7032 non-null   object 
 13  StreamingTV       7032 non-null   object 
 14  StreamingMovies   7032 non-null   object 
 15  Contract          7032 non-null   object 
 16  PaperlessBilling  7032 non-null   object 


<ipython-input-90-fd8a457607dc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_churn_data.TotalCharges = cleaned_churn_data.TotalCharges.astype(float)


Seperating the target variable.

In [91]:
y = cleaned_churn_data.Churn

In [92]:
cleaned_churn_data.drop('Churn', axis=1, inplace=True)
print(cleaned_churn_data.info())
print(y)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   gender            7032 non-null   object 
 2   SeniorCitizen     7032 non-null   int64  
 3   Partner           7032 non-null   object 
 4   Dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   PhoneService      7032 non-null   object 
 7   MultipleLines     7032 non-null   object 
 8   InternetService   7032 non-null   object 
 9   OnlineSecurity    7032 non-null   object 
 10  OnlineBackup      7032 non-null   object 
 11  DeviceProtection  7032 non-null   object 
 12  TechSupport       7032 non-null   object 
 13  StreamingTV       7032 non-null   object 
 14  StreamingMovies   7032 non-null   object 
 15  Contract          7032 non-null   object 
 16  PaperlessBilling  7032 non-null   object 


<ipython-input-92-ff32e38a10b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_churn_data.drop('Churn', axis=1, inplace=True)


Spliting data to train and to validate.

In [93]:
from sklearn.model_selection import train_test_split

X = cleaned_churn_data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1, train_size=0.8, test_size=0.2)


The first model 